# 🔧 Environment Setup & Verification

This notebook verifies that all required libraries are installed and working correctly.

## Objectives
- Import and verify all required libraries
- Check TensorFlow, OpenCV, and MediaPipe installations
- Verify GPU availability (if applicable)
- Set up plotting configurations
- Test webcam access

---

## 1. Import Core Libraries

In [ ]:
# Core Python libraries
import os
import sys
import warnings
warnings.filterwarnings('ignore')

print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")

## 2. Import Data Science Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

print("✅ NumPy version:", np.__version__)
print("✅ Pandas version:", pd.__version__)
print("✅ Matplotlib version:", plt.matplotlib.__version__)

## 3. Import Computer Vision Libraries

In [ ]:
import cv2
import mediapipe as mp

print("✅ OpenCV version:", cv2.__version__)
print("✅ MediaPipe version:", mp.__version__)

## 4. Import Deep Learning Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

print("✅ TensorFlow version:", tf.__version__)
print("✅ Keras version:", keras.__version__)

## 5. Import Machine Learning Utilities

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn

print("✅ Scikit-learn version:", sklearn.__version__)

## 6. Check GPU Availability

In [ ]:
# Check for GPU
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("\n" + "="*60)
    print("🎉 GPU DETECTED! Training will be accelerated.")
    print("="*60)
    for gpu in gpus:
        print(f"  GPU: {gpu}")
        # Enable memory growth to prevent TensorFlow from allocating all GPU memory
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("  Memory growth enabled")
        except RuntimeError as e:
            print(f"  Error setting memory growth: {e}")
else:
    print("\n" + "="*60)
    print("⚠️  No GPU detected. Training will use CPU (slower).")
    print("    This is fine for small datasets and prototyping.")
    print("="*60)

## 7. Configure Plotting Settings

In [ ]:
# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set default figure parameters
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✅ Plotting configuration set")

## 8. Set Random Seeds for Reproducibility

In [ ]:
# Set random seeds
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"✅ Random seed set to {RANDOM_SEED} for reproducibility")

## 9. Verify Directory Structure

In [ ]:
# Define project directories
PROJECT_DIRS = [
    'data/raw',
    'data/processed',
    'models/saved_models',
    'outputs/logs',
    'outputs/metrics',
    'outputs/visualizations',
    'notebooks'
]

# Check if directories exist
print("\nDirectory Structure:")
print("="*60)

for directory in PROJECT_DIRS:
    if os.path.exists(directory):
        print(f"✅ {directory}")
    else:
        print(f"❌ {directory} (missing)")
        os.makedirs(directory, exist_ok=True)
        print(f"   Created: {directory}")

print("="*60)

## 10. Test Webcam Access

In [ ]:
# Test webcam
print("Testing webcam access...")
print("A window should open showing your webcam feed.")
print("Press any key to close the window.\n")

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ ERROR: Cannot access webcam!")
    print("   Please check:")
    print("   1. Webcam is connected")
    print("   2. No other application is using the webcam")
    print("   3. Camera permissions are granted")
else:
    print("✅ Webcam access successful!")
    
    # Capture a test frame
    ret, frame = cap.read()
    
    if ret:
        print(f"   Frame size: {frame.shape[1]}x{frame.shape[0]}")
        print(f"   Channels: {frame.shape[2]}")
        
        # Display frame
        cv2.putText(frame, "Webcam Test - Press any key to close", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow('Webcam Test', frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
cap.release()
cv2.destroyAllWindows()

## 11. Test MediaPipe Hand Detection

In [ ]:
# Test MediaPipe
print("Testing MediaPipe hand detection...")
print("Show your hand to the camera.")
print("Press ESC to close.\n")

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands:
    
    frame_count = 0
    detection_count = 0
    
    while cap.isOpened() and frame_count < 100:  # Test for 100 frames
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            detection_count += 1
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )
        
        cv2.putText(frame, "MediaPipe Test - Press ESC to close", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Detections: {detection_count}/{frame_count}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('MediaPipe Hand Detection Test', frame)
        
        frame_count += 1
        
        if cv2.waitKey(1) & 0xFF == 27:  # ESC
            break

cap.release()
cv2.destroyAllWindows()

if detection_count > 0:
    print(f"\n✅ MediaPipe working correctly!")
    print(f"   Detected hands in {detection_count}/{frame_count} frames")
else:
    print(f"\n⚠️  No hands detected. Make sure to show your hand to the camera.")

## 12. System Summary

In [ ]:
print("\n" + "="*60)
print("ENVIRONMENT SETUP COMPLETE")
print("="*60)
print("\n📦 Installed Packages:")
print(f"  • Python: {sys.version.split()[0]}")
print(f"  • TensorFlow: {tf.__version__}")
print(f"  • OpenCV: {cv2.__version__}")
print(f"  • MediaPipe: {mp.__version__}")
print(f"  • NumPy: {np.__version__}")
print(f"  • Pandas: {pd.__version__}")
print(f"  • Scikit-learn: {sklearn.__version__}")

print("\n🖥️  Hardware:")
if gpus:
    print(f"  • GPU: Available ({len(gpus)} device(s))")
else:
    print(f"  • GPU: Not available (using CPU)")

print("\n✅ You're ready to proceed to the next notebook!")
print("   Next: 02_data_collection.ipynb")
print("="*60)

---

## 🎯 Next Steps

If all tests passed successfully, proceed to:
- **02_data_collection.ipynb** - Collect sign language gesture data

If you encountered any errors:
1. Check that all packages are installed: `pip install -r requirements.txt`
2. Verify webcam permissions
3. Restart the kernel and try again

---